# 2. Limpieza y pre-procesamiento

In [1]:
# manipulacion archivos
import os

# manipulacion datos
import numpy as np
import pandas as pd

## 2.1. Limpieza de datos

### 2.1.1. Datos de CoinGecko
Empezamos con los datos extraidos de la plataforma CoinGecko

In [2]:
data_market = pd.read_pickle(os.path.join("1_data","data_market.pkl"))
data_market.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34099 entries, 2072 to 56288
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   coin_id        34099 non-null  object 
 1   prices         34099 non-null  float64
 2   year           34099 non-null  int32  
 3   month          34099 non-null  int32  
 4   day            34099 non-null  int32  
 5   market_caps    34099 non-null  float64
 6   total_volumes  34099 non-null  float64
 7   category_id    34099 non-null  object 
 8   categories     34099 non-null  object 
dtypes: float64(3), int32(3), object(3)
memory usage: 2.2+ MB


#### 2.1.1.1. Duplicados y valores faltantes
En este seccion verificamos si hay filas duplicadas y valores faltantes.

Primero, identificamos filas duplicadas. Nos concentramos en las variables cuantitivas y nuestra variable categoricas 'category_id' y 'coin_id'; excluimos la variable que contiene informacion cualitativa en formato lista de lista 'categories' para facilitar la identificacion de duplicados; esta exclusion no afecta el principal objetivo de identificar duplicados: garantizar que cada fila contiene informacion unica. En los resultados de abajo se puede observar que no existen filas duplicadas y, por tanto, todas las filas contienen informacion unica.

In [3]:
data_market_notuple = data_market.loc[:, data_market.columns != 'categories']
data_market_notuple.loc[data_market_notuple.duplicated(keep=False)]

,coin_id,prices,year,month,day,market_caps,total_volumes,category_id


Segundo, identificamos valores nulos. En los resultados de abajo se puede observar que no existen valores faltantes.

In [4]:
data_market.isna().sum()

coin_id          0
prices           0
year             0
month            0
day              0
market_caps      0
total_volumes    0
category_id      0
categories       0
dtype: int64

### 2.1.1.2. Outliers
En esta sección, identificamos los outliers haciendo uso del criterio de los 3 sigmas (z-score). Empleamos el z-score por moneda, año y mes. Esperamos obtener varios outliers dada la volatilidad del mercado de criptomonedas ampliamente documentada en la literatura. Por tal motivo, no creemos pertinente removerlos o imputarlos puesto que consideramos que estos valores no son datos invalidos; es decir, confiamos en la validez de nuestra fuente de datos y consideramos que los outliers identifcados abajo son observaciones que hacen parte de la población y por tanto NO deben ser ni removidos ni imputados.

In [5]:
# funcion 3 sigmas
def outliers_zscore(x):
    z_score = np.abs((x - np.mean(x)) / np.std(x))
    return z_score > 3

Empezamos con los datos extraidos de CoinGecko. En los resultados de abajo se puede observar que hay un maximo de 2 observaciones atipicas en 'prices' para cada moneda-año-mes y que en total son 173 observaciones atipicas. Nuevamente, los casos atpicos son inherentes a cualquier poblacion, incluyendo poblaciones con una distribucion normal. Por tal motivo, ni se remueven ni se imputan.

In [6]:
# empleamos el z-score por moneda, año y mes
data_market[['prices_outliers']]=data_market.groupby(['coin_id','year','month'])[['prices']].transform(outliers_zscore)

# contamos lo casos atipicos.
print(data_market[['coin_id','year','month']].loc[data_market['prices_outliers']>=1].value_counts().sort_values(ascending=False))
print("Total de outliers:", data_market[['coin_id','year','month']].loc[data_market['prices_outliers']>=1].value_counts().sum())

coin_id         year  month
bitcoin         2019  4        2
stasis-eurs     2021  12       2
gemini-dollar   2023  3        2
paxos-standard  2023  8        2
bitcoin-cash    2019  4        2
                              ..
usd-coin        2022  12       1
                2023  2        1
                      3        1
                      6        1
                      8        1
Name: count, Length: 166, dtype: int64
Total de outliers: 173


En los resultados de abajo se puede observar que hay un maximo de 2 observaciones atipicas en 'total_volumes' para cada moneda-año-mes y que en total son 173 observaciones atipicas. Nuevamente, los casos atpicos son inherentes a cualquier poblacion, incluyendo poblaciones con una distribucion normal. Por tal motivo, ni se remueven ni se imputan.

In [7]:
# empleamos el z-score por moneda, año y mes
data_market[['volumes_outliers']]=data_market.groupby(['coin_id','year','month'])[['total_volumes']].transform(outliers_zscore)

# contamos lo casos atipicos.
print(data_market[['coin_id','year','month']].loc[data_market['volumes_outliers']>=1].value_counts().sort_values(ascending=False))
print("Total de outliers:", data_market[['coin_id','year','month']].loc[data_market['volumes_outliers']>=1].value_counts().sum())

coin_id           year  month
fantom            2020  1        2
crypto-com-chain  2023  8        2
paxos-standard    2019  2        2
fantom            2019  8        2
stasis-eurs       2020  3        2
                                ..
usd-coin          2021  7        1
                        10       1
                        12       1
                  2022  1        1
                  2023  8        1
Name: count, Length: 473, dtype: int64
Total de outliers: 496


In [8]:
# restauramos nuestro dataframe
data_market=data_market.drop(columns=['prices_outliers','volumes_outliers'])
data_market.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34099 entries, 2072 to 56288
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   coin_id        34099 non-null  object 
 1   prices         34099 non-null  float64
 2   year           34099 non-null  int32  
 3   month          34099 non-null  int32  
 4   day            34099 non-null  int32  
 5   market_caps    34099 non-null  float64
 6   total_volumes  34099 non-null  float64
 7   category_id    34099 non-null  object 
 8   categories     34099 non-null  object 
dtypes: float64(3), int32(3), object(3)
memory usage: 2.2+ MB


### 2.1.2. Datos del Banco Mundial (WB)
Ahora nos concentramos en los datos extraidos del WB.

In [9]:
data_deflators = pd.read_pickle(os.path.join("1_data","data_deflators.pkl"))
data_deflators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   code_pais       40 non-null     object 
 1   year            40 non-null     int64  
 2   gdpdfltr_2015   40 non-null     float64
 3   pppfactor_2015  40 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.4+ KB


#### 2.1.2.1. Duplicados y valores faltantes
En este seccion verificamos si hay filas duplicadas y valores faltantes.

Primero, identificamos filas duplicadas. En los resultados de abajo se puede observar que no existen filas duplicadas y, por tanto, todas las filas contienen informacion unica.

In [10]:
data_deflators.loc[data_deflators.duplicated(keep=False)]

,code_pais,year,gdpdfltr_2015,pppfactor_2015


Segundo, identificamos valores nulos. En los resultados de abajo se puede observar que no existen valores faltantes.

In [11]:
data_deflators.isna().sum()

code_pais         0
year              0
gdpdfltr_2015     0
pppfactor_2015    0
dtype: int64

### 2.1.2.2. Outliers
En esta sección, identificamos los outliers haciendo uso del criterio de los 3 sigmas (z-score). Empleamos el z-score por pais. Como en el caso anterior, confiamos en la validez de nuestra fuente de datos y consideramos que cualquier outlier constituyen observaciones que hacen parte de la población y por tanto NO deben ser ni removidos ni imputados.

Los resultado de abajo muestra que no existen observaciones atipicas.

In [12]:
# empleamos el z-score por moneda, año y mes
data_deflators[['gdpdfltr_2015_outliers']]=data_deflators.groupby(['code_pais'])[['gdpdfltr_2015']].transform(outliers_zscore)

# contamos lo casos atipicos.
print(data_deflators[['code_pais']].loc[data_deflators['gdpdfltr_2015_outliers']>=1].value_counts().sort_values(ascending=False))
print("Total de outliers:", data_deflators[['code_pais']].loc[data_deflators['gdpdfltr_2015_outliers']>=1].value_counts().sum())

Series([], Name: count, dtype: int64)
Total de outliers: 0


In [13]:
# empleamos el z-score por moneda, año y mes
data_deflators[['pppfactor_2015_outliers']]=data_deflators.groupby(['code_pais'])[['pppfactor_2015']].transform(outliers_zscore)

# contamos lo casos atipicos.
print(data_deflators[['code_pais']].loc[data_deflators['pppfactor_2015_outliers']>=1].value_counts().sort_values(ascending=False))
print("Total de outliers:", data_deflators[['code_pais']].loc[data_deflators['pppfactor_2015_outliers']>=1].value_counts().sum())

Series([], Name: count, dtype: int64)
Total de outliers: 0


In [14]:
# restauramos nuestro dataframe
data_deflators=data_deflators.drop(columns=['gdpdfltr_2015_outliers','pppfactor_2015_outliers'])
data_deflators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   code_pais       40 non-null     object 
 1   year            40 non-null     int64  
 2   gdpdfltr_2015   40 non-null     float64
 3   pppfactor_2015  40 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.4+ KB


## 2.2. Pre-procesamiento: Volatilidad relativa en precios corrientes y constantes

Esta seccion la dedicamos al pre-procesamiento de datos. La dividimos en dos partes.

En la primera parte, nos concentramos en calcular nuestro primer indicador (KPI) "Volatilidad relativa" en el corto plazo. Esto lo hacemos con base en precios corrientes en USD$ de CoinGecko. El calculo de la "Volatilidad relativa" los hacemos en dos pasos.

En el primer paso, calculamos la desviacion estandar con el fin de medir variacion en los precios corrientes de cada moneda por año. Concentrarnos en la variacion de cada moneda por año tiene la virtud que controlamos por el efect to la inflacion entre año 1 y año 2; por esta razon, esta medida nos indica la variacion en el corto plazo - es decir, en el año.

El segundo paso consiste en relacionar el concepto de variacion al de volatilidad; es decir, ¿como sabemos si la variacion observado puede considerarse volatil? Para responder a esta pregunta, medimos la proporcion entre la volatilidad de cada moneda-año y la volatilidad promedio de todas las otras monedas en el año; es decir, desv. std. (moneda por año) ÷ desv. std. (todas las monedas por año). De esta manera, medimos la magnitud de la volatilidad (desv. std.) de cada moneda en relacion a la volatilidad (desv. std.) de todas las monedas en un mismo año; por esta razon, denominamos este KPI como "Volatilidad relativa" en el corto plazo, por que nuestra unidad de tiempo es el año. 

En la segunda parte, nos concentramos en calcular nuestro segundo indicador (KPI) "Volatilidad local relativa" en el mediano plazo; es decir, es similar al anterior, pero varia en terminos que hace referencia a la volatlidad segun el contexto de cada pais. En este caso tambien hacemos uso de la desviacion estandar para medir la magnitud de la volatilidad de cada moneda en relacion al promedio de todas las monedas para el mismo año. Sin embargo, lo hacemos con base en una serie de tiempo en precios constantes y ajustada de acuerdo a la paridad del poder adquisitivo para cuatro paises Latino-Americanos y EEUU. Como mencionamos en la introduccion, estamos interesado en medir si la volatilidad inherente a las criptomonedas se ve intensificada en contextos de cambios en la inflacion local junto con volatilidad de la tasa cambiaria que caracterizan a paises de Latino-America. Damos cuanta de estos en dos pasos. Primero, convertimos los precios corrientes de Coigecko a precios constante usando indices precios que construimos con base en nuestros datos extraidos del Banco Mundial; obtener precios constantes permite controlar por cambios en la inflacion y, por tal motivo, pasar de una Volatilidad relativa en el corto plazo a una en el mediano plazo. Segundo, expresamos estos precios constantes en precios que relfejen una paridad del poder adquisitivo; de esta manera, obtenemos precios internacionales pero ajustados segun el poder adquisitivo de cada pais (ver [ref](https://es.wikipedia.org/wiki/Paridad_de_poder_adquisitivohttps://es.wikipedia.org/wiki/Paridad_de_poder_adquisitivo)). De esta manera, nuestra Volatilidad relativa se convierte en una "Volatilidad local relativa". Esto nos permite comparar la volatilidad del mercado de criptomonedas entre paises. 

### 2.2.1. Volatilidad relativa en el corto plazo: precios corrientes

Primero, calculamos la desviacion estandar promedio de todas las monedas para cada año.

In [15]:
data_market_prcvoltlty_group_mean = pd.DataFrame(
    data_market.groupby(['category_id','coin_id','year'])['prices'].std().reset_index()
    .groupby(['category_id','year'])['prices'].mean()
    ).rename(columns={'prices':'sd_group_mean'}).reset_index()
data_market_prcvoltlty_group_mean

,category_id,year,sd_group_mean
0,layer-1,2019,255.734510
1,layer-1,2020,402.039620
2,layer-1,2021,1023.298805
3,layer-1,2022,1018.478820
4,layer-1,2023,347.102606
5,stablecoins,2019,2.822968
6,stablecoins,2020,14.540918
7,stablecoins,2021,5.314946
8,stablecoins,2022,9.663229
9,stablecoins,2023,6.188310


Segundo, calculamos la desviacion estandar promedio cada cada moneda para cada año.

In [16]:
data_market_prcvoltlty_coin = pd.DataFrame(
    data_market.groupby(['category_id','coin_id','year'])['prices'].std()
    ).rename(columns={'prices':'sd_coin'}).reset_index()

Tercero, relacionamos la desviacion estandar de cada moneda con el promedio de todas las monedas para el mismo año; de esta manera obtenemos nuestro KPI "Volatilidad relativa" `vltlty_ratio`.

Cuarto, rankeamos las monedas en terminos de las mas volatil a la menos volatil segun nuestro KIP "Volatilidad relativa" `vltlty_ratio`. Este ranking `vltlty_rank` nos servira para mas adelante en nuestro analisis.

In [17]:
data_market_prcvoltlty_crrnt = data_market_prcvoltlty_coin.merge(
    data_market_prcvoltlty_group_mean,
    how='left')
data_market_prcvoltlty_crrnt[
    'vltlty_ratio'] = data_market_prcvoltlty_crrnt[
        'sd_coin'] / data_market_prcvoltlty_crrnt['sd_group_mean']
data_market_prcvoltlty_crrnt[
    'vltlty_rank'] = data_market_prcvoltlty_crrnt.groupby(
        ['category_id','year'])['vltlty_ratio'].rank(method='dense', ascending=False)
data_market_prcvoltlty_crrnt.sort_values(
    by=['category_id','year','vltlty_rank'],
    kind='stable',
    inplace=True)
data_market_prcvoltlty_crrnt.to_pickle(
    os.path.join("2_pipeline","data_market_prcvoltlty_crrnt.pkl"))
data_market_prcvoltlty_crrnt.head(11)

,category_id,coin_id,year,sd_coin,sd_group_mean,vltlty_ratio,vltlty_rank
10,layer-1,bitcoin,2019,2655.712480,255.73451,10.384646,1.0
15,layer-1,bitcoin-cash,2019,96.242064,255.73451,0.376336,2.0
35,layer-1,ethereum,2019,50.775028,255.73451,0.198546,3.0
5,layer-1,binancecoin,2019,8.374184,255.73451,0.032746,4.0
25,layer-1,cosmos,2019,1.132393,255.73451,0.004428,5.0
0,layer-1,algorand,2019,0.411165,255.73451,0.001608,6.0
50,layer-1,tezos,2019,0.372936,255.73451,0.001458,7.0
30,layer-1,crypto-com-chain,2019,0.021568,255.73451,0.000084,8.0
20,layer-1,cardano,2019,0.018532,255.73451,0.000072,9.0
45,layer-1,hedera-hashgraph,2019,0.012550,255.73451,0.000049,10.0


## 2.2. Volatilidad relativa local en el mediano plazo: precios constantes y ajustados PPP
En la primera seccion, deflacionamos los precios corrientes en precios constantantes y los expresamos en precios segun su paridad del poder adquisitivo (PPP).

En la segunda seccion, medimos la "Volatilidad relativa local" en el mediano plazo. Esto consiste en, primero, medir la volatilidad de cada moneda en cada año y pais por medio de su desviacion estandar. Segundo, medimos el promedio de todas las monedas para cada año por pais; es decir, el promedio anual de volatilidad por pais. Como los precios esta expresado en precios constantes, podemos comparar los promedios anuales de volatilidad entre paises sin riesgo de preocuparnos que estamos ignorando cambios por la inflacion.

### 2.2.1. Transformacion a precios constantes ajustados segun paridad del poder adquisitivo

Unimos nuestros datos.

In [18]:
data_market_constant = data_market.merge(
    data_deflators,
    on='year'
)
data_market_constant.info()
data_market_constant.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144875 entries, 0 to 144874
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   coin_id         144875 non-null  object 
 1   prices          144875 non-null  float64
 2   year            144875 non-null  int32  
 3   month           144875 non-null  int32  
 4   day             144875 non-null  int32  
 5   market_caps     144875 non-null  float64
 6   total_volumes   144875 non-null  float64
 7   category_id     144875 non-null  object 
 8   categories      144875 non-null  object 
 9   code_pais       144875 non-null  object 
 10  gdpdfltr_2015   144875 non-null  float64
 11  pppfactor_2015  144875 non-null  float64
dtypes: float64(5), int32(3), object(4)
memory usage: 11.6+ MB


,coin_id,prices,year,month,day,market_caps,total_volumes,category_id,categories,code_pais,gdpdfltr_2015,pppfactor_2015
0,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",ARG,78.354031,0.685846
1,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",BRA,102.945851,0.597797
2,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",COL,100.505932,0.465566
3,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",MEX,101.312482,0.525458
4,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",USA,107.285879,1.000000
5,bitcoin,3794.264254,2019,1,2,6.618845e+10,2.689878e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",ARG,78.354031,0.685846
6,bitcoin,3794.264254,2019,1,2,6.618845e+10,2.689878e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",BRA,102.945851,0.597797
7,bitcoin,3794.264254,2019,1,2,6.618845e+10,2.689878e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",COL,100.505932,0.465566
8,bitcoin,3794.264254,2019,1,2,6.618845e+10,2.689878e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",MEX,101.312482,0.525458
9,bitcoin,3794.264254,2019,1,2,6.618845e+10,2.689878e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",USA,107.285879,1.000000


Primero, convertimos la serie de tiempo en precios corrientes a precios constantes usando nuestro indices de precios. Segundo, ajustamos los precios para que reflejen una paridad en el poder adquisitivo. 

In [19]:
data_market_constant[
    'prices_cnstnt2015'] = data_market_constant[
        'prices'] * (100/data_market_constant['gdpdfltr_2015'])
data_market_constant[
    'prices_ppp2015']=data_market_constant[
        'prices_cnstnt2015']/data_market_constant['pppfactor_2015']
data_market_constant.head()

,coin_id,prices,year,month,day,market_caps,total_volumes,category_id,categories,code_pais,gdpdfltr_2015,pppfactor_2015,prices_cnstnt2015,prices_ppp2015
0,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",ARG,78.354031,0.685846,4712.624899,6871.263049
1,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",BRA,102.945851,0.597797,3586.867784,6000.141521
2,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",COL,100.505932,0.465566,3673.943900,7891.355132
3,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",MEX,101.312482,0.525458,3644.695576,6936.229175
4,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",USA,107.285879,1.000000,3441.768494,3441.768494


### 2.2.2. Volatilidad relativa local en el mediano plazo

Primero, medimos la volatilidad de cada moneda por año y pais con base en nuestro df creados en el paso anterior. Segundo, medimos el promedio anual de volatilidad por pais.

In [20]:
data_market_constant_prcvoltlty_group_mean = pd.DataFrame(
    data_market_constant.groupby(
        ['category_id','code_pais','coin_id','year'])['prices_ppp2015'].std().reset_index()
    .groupby(['category_id','code_pais','year'])['prices_ppp2015'].mean()
    ).rename(columns={'prices_ppp2015':'sd_group_pais_mean'}).reset_index()
data_market_constant_prcvoltlty_group_mean.head(8)

,category_id,code_pais,year,sd_group_pais_mean
0,layer-1,ARG,2019,475.884650
1,layer-1,ARG,2020,782.470174
2,layer-1,ARG,2021,1739.810852
3,layer-1,ARG,2022,1403.234830
4,layer-1,BRA,2019,415.553185
5,layer-1,BRA,2020,801.906317
6,layer-1,BRA,2021,1917.492417
7,layer-1,BRA,2022,1687.184809


Hacemos lo mismo pero con la mediana; es decir, la mediana anual de volatilidad por pais.

In [21]:
data_market_constant_prcvoltlty_group_median = pd.DataFrame(
    data_market_constant.groupby(
        ['category_id','code_pais','coin_id','year'])['prices_ppp2015'].std().reset_index()
    .groupby(['category_id','code_pais','year'])['prices_ppp2015'].median()
    ).rename(columns={'prices_ppp2015':'sd_group_pais_median'}).reset_index()
data_market_constant_prcvoltlty_group_median.head(8)

,category_id,code_pais,year,sd_group_pais_median
0,layer-1,ARG,2019,0.765119
1,layer-1,ARG,2020,1.204551
2,layer-1,ARG,2021,2.542353
3,layer-1,ARG,2022,1.455064
4,layer-1,BRA,2019,0.668119
5,layer-1,BRA,2020,1.234471
6,layer-1,BRA,2021,2.801995
7,layer-1,BRA,2022,1.749502


Unimos los dfs.

In [22]:
data_market_constant_prcvoltlty_coin = pd.DataFrame(
    data_market_constant.groupby(
        ['category_id','code_pais','coin_id','year'])['prices_ppp2015'].std()
    ).rename(columns={'prices_ppp2015':'sd_pais_coin'}).reset_index()
data_market_prcvoltlty_constant = data_market_constant_prcvoltlty_coin.merge(
    data_market_constant_prcvoltlty_group_mean,
    how='left').merge(
        data_market_constant_prcvoltlty_group_median,
        how='left')
data_market_prcvoltlty_constant.to_pickle(
    os.path.join("2_pipeline","data_market_prcvoltlty_constant.pkl"))
data_market_prcvoltlty_constant.head(11)

,category_id,code_pais,coin_id,year,sd_pais_coin,sd_group_pais_mean,sd_group_pais_median
0,layer-1,ARG,algorand,2019,0.765119,475.884650,0.765119
1,layer-1,ARG,algorand,2020,0.193741,782.470174,1.204551
2,layer-1,ARG,algorand,2021,0.758258,1739.810852,2.542353
3,layer-1,ARG,algorand,2022,0.463004,1403.234830,1.455064
4,layer-1,ARG,binancecoin,2019,15.583135,475.884650,0.765119
5,layer-1,ARG,binancecoin,2020,12.100110,782.470174,1.204551
6,layer-1,ARG,binancecoin,2021,289.322226,1739.810852,2.542353
7,layer-1,ARG,binancecoin,2022,99.495651,1403.234830,1.455064
8,layer-1,ARG,bitcoin,2019,4941.893849,475.884650,0.765119
9,layer-1,ARG,bitcoin,2020,8201.913436,782.470174,1.204551
